# Revised case normalization for Hirslanden Birshof 2017

Notes:

- 4 revised DtoD cases were joined with the BFS cases from the DB.



In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI_Aarau_Birshof_ST. Anna.xlsx'),
                     'Hirslanden Birshof', '2017', 'Birshof_2017')

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI_Aarau_Birshof_ST. Anna.xlsx', hospital_name_db='Hirslanden Birshof', year='2017', sheet='Birshof_2017')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-10 12:02:04.760 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:35 - Read 4 cases for Hirslanden Birshof 2017
2022-11-10 12:02:04.763 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:59 - TYPES:
datum                                                string
tranche                                              string
case_id                                              object
patient_id                                           object
gender                                               string
age_years                                             int64
bfs_code                                             string
duration_of_stay                                      int64
pflegetage neu                                       string
pccl                                                  int64
pccl neu                                             string
old_pd                                               string
primary_

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,0041497128,610ECC6F4184EF0C,M,67,7,41497128,M4807,M4807,M200,3,I09D,"[M4036, D62]",[],[],"[009430::20171027, 009420::20171027]"
1,0041319879,F76D75CD0067F6E4,W,72,5,41319879,M4726,M4726,M200,2,I09E,[L8915],[],[],[]
2,0041332448,9BAAC9A58B5FB056,W,57,12,41332448,T845,T845,M200,3,I46C,[I8080],[],[],[]
3,0041371186,AED552F02ADC590F,M,59,22,41371186,T845,T845,M200,3,I03B,[N182],[],[],[5794::20170531]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases

2022-11-10 12:02:05.933 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:90 - Read 3653 rows from the DB, for the hospital 'Hirslanden Birshof' in 2017


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,20243,0041497128,M4807,"[G553, M4217, M4787, M513, D6980, ...]",8107::20171026,"[889714::20171026, 030939::20171026, 784986::20171026, 845111::20171026, 805111::20171026, ...]",M,67,0,0,7,0,01,2017-10-26,0,00,2017-11-02
1,20498,0041332448,T845,"[Y849, B956, I1090, I8080]",8153:L:20170328,"[803510:L:20170327, 887951::20170327, 819231:L:20170328]",W,57,0,0,12,0,01,2017-03-26,0,00,2017-04-07
2,21005,0041319879,M4726,"[G552, G9581, M4316, I1090, K219, ...]",8108::20170322,"[816210::20170322, 030934::20170322, 805110::20170322, 845110::20170322, 785035::20170322, ...]",W,72,0,0,5,0,01,2017-03-22,0,00,2017-03-27
3,22492,0041371186,T845,"[Y828, B957, T840, I1190, E1190, ...]",815121:R:20170518,"[800515:R:20170518, 776520:R:20170518, 780512:R:20170518, 890A12::20170518, 992214::20170519, ...]",M,59,0,0,22,0,01,2017-05-18,0,00,2017-06-09


In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-10 12:02:06.474 | INFO     | src.revised_case_normalization.notebook_functions.group:group:83 - Grouping 4 cases ...
2022-11-10 12:02:07.905 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:88 - Grouped 4 cases into: 4 revisions, 39 diagnoses rows, 42 procedure rows


In [7]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,20243,I06B,5.16,5.16,4,2022-12-31
1,20498,I46A,2.66,2.66,4,2022-12-31
2,21005,I09A,3.87,3.87,4,2022-12-31
3,22492,I03A,4.28,4.28,4,2022-12-31


In [8]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-10 12:02:07.930 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:249 - Trying to insert 4 cases into the 'Revisions' table ...
2022-11-10 12:02:08.736 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:275 - Deleted 4 rows from the "Revisions" table, which is about to be updated
2022-11-10 12:02:08.834 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:286 - Inserted 4 cases into the 'Revisions' table
2022-11-10 12:02:08.836 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:298 - Trying to insert 39 rows into the 'Diagnoses' table ...
2022-11-10 12:02:10.332 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:336 - Inserted 39 rows into the 'Diagnoses' table
2022-11-10 12:02:10.334 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:347 - Trying to insert 42 rows into the 'Procedures' table ..